# Makes common margins - from options closest to undPrice and with earliest strike

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "NSE"

# Set the root
from from_root import from_root

ROOT = from_root()

import pandas as pd
from loguru import logger

pd.options.display.max_columns = None
pd.set_option("display.precision", 2)

import sys
from pathlib import Path

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util

util.startLoop()

logger.add(sink=ROOT / "log" / "ztest.log", mode="w")

1

# Imports

In [2]:
from ib_async import IB
from tqdm import tqdm

from ibfuncs import marginsAsync
from nse import NSEfnos, equity_iv_df
from utils import append_cos, get_closest_strike, pickle_me, chunk_me, load_config

config = load_config()
port = config.get('PORT')

# Make the symbols

In [3]:
# TRANSFERRED TO NSE.PY

def get_all_fno_names() -> set:
    """All fnos in nse, including index"""

    n = NSEfnos()
    d = n.stock_quote_fno('NIFTY')
    fnos = n.equities() | set(d.get('allSymbol'))
    return fnos

In [4]:
# TRANSFERRED TO NSE.PY

def make_raw_fno_df(fnos) -> pd.DataFrame:
    """Makes all the raw fnos"""

    n = NSEfnos()

    dfs = []

    with tqdm(total=len(fnos), desc='Generating raw fnos', unit='s') as pbar:
        
        for s in tqdm(fnos):
            try:
                df = equity_iv_df(n.stock_quote_fno(s))
                dfs.append(df)
            except Exception as e:
                logger.error(f"Error for {s} - error: {e}")
                pass

            pbar.update(1)
            
    df = pd.concat(dfs, ignore_index=True)

    return df

## Get the closest strikes

In [5]:
def update_margin_comm(df: pd.DataFrame, dfm: pd.DataFrame) -> pd.DataFrame:
    """
    Update the 'margin' and 'comm' columns in the given DataFrame from another DataFrame.

    This function updates the 'margin' and 'comm' columns in the input DataFrame `df`
    using the values from `dfm`. If the 'margin' and 'comm' columns do not exist in
    `df`, they will be added. The update is based on matching 'ib_symbol' values.

    Args:
        df (pd.DataFrame): The original DataFrame containing at least an 'ib_symbol' column.
        dfm (pd.DataFrame): A DataFrame containing 'ib_symbol', 'margin', and 'comm' columns
                            with new values to update in `df`.

    Returns:
        pd.DataFrame: The updated DataFrame with 'margin' and 'comm' columns updated or added.
    """
    
    # Set 'ib_symbol' as the index for both DataFrames to align them
    df1 = df.set_index('ib_symbol')
    df2 = dfm[['ib_symbol', 'margin', 'comm']].set_index('ib_symbol')

    # Update existing columns
    df1.update(df2)

    # Add new columns if they do not exist
    for col in ['margin', 'comm']:
        if col in df2.columns:
            df1[col] = df2[col]

    # Reset index to bring 'ib_symbol' back as a column
    df1.reset_index(inplace=True)

    return df1

In [7]:
## Cleand up by chatgpt 4o
def get_closest_margins(port: int, save: bool = False, timeout: int = 3):
    """Gets margins of strikes closest to undPrice with earliest expiry"""

    # Retrieve all FNO names and create a raw DataFrame
    df = make_raw_fno_df(get_all_fno_names())
    
    # Get closest strikes by grouping and applying the closest strike function
    dfe = df.loc[df.groupby("ib_symbol").apply(get_closest_strike).index]
    
    # Append the cost of sales information and chunk the DataFrame
    df_chunks = chunk_me(append_cos(dfe)[['contract', 'order']], 10)
    
    # Initialize a list to store margins from each chunk
    dfs = []

    # Use a progress bar to track the processing of chunks
    with tqdm(total=len(df_chunks), desc="Getting closest strike margins", unit="symbol") as pbar:
        for dfo in df_chunks:
            with IB().connect(port=port) as ib:
                # Get margins asynchronously and append to the list
                dfs.append(ib.run(marginsAsync(ib=ib, df=dfo, timeout=timeout)))
                pbar.update(1)
    
    # Concatenate all margins into a single DataFrame
    df_mcom = pd.concat(dfs, ignore_index=True)
    
    # Clean up margins and commissions
    df_margins = pd.concat([dfe.reset_index(drop=True), df_mcom], axis=1)
    df_margins['comm'] = np.where(df_margins.comm.isnull(), df_margins.comm.max(), df_margins.comm)
    df_margins['margin'] = np.where(df_margins.margin <= 0, np.nan, df_margins.margin)

    # Update the original DataFrame with new margins and commissions
    df_all = update_margin_comm(df, df_margins)

    # Save the DataFrames if the save flag is set
    if save:
        pickle_me(df_all, ROOT / 'data' / 'df_all_opts.pkl')
        pickle_me(df_margins, ROOT / 'data' / 'df_und_margins.pkl')
        
    return df_margins

In [17]:
# Update the original DataFrame with new margins and commissions
df_all = update_margin_comm(df, df_margins)

NameError: name 'df' is not defined

In [ ]:
update_margin_comm(df, dfm)

In [ ]:
df.head()

In [ ]:
dfm.head()

In [ ]:
df1 = df.set_index('ib_symbol')
df2 = dfm[['ib_symbol', 'margin', 'comm']].set_index('ib_symbol')

df1.update(df2)

df1

In [ ]:
df2